In [2]:
import sys
import os

# Current notebook folder: win_tracker/soccer
notebook_dir = os.getcwd()

# Project root for imports: win_tracker/
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))

# Add to sys.path if not already there
if project_root not in sys.path:
    sys.path.append(project_root)

print("Project root added to sys.path:", project_root)

# Now you can import
from func import add_winner_column
from func import map_teams


from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, date_format, when



Project root added to sys.path: /Users/Jake/Documents/GitHub/all-sports-win-tracker/win_tracker


In [3]:
spark = SparkSession.builder.appName("FPL").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/23 14:39:49 WARN Utils: Your hostname, MacBook-Air-197.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.97 instead (on interface en0)
25/11/23 14:39:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/23 14:39:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/23 14:39:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
# Read in prior data
py_csv_file_path = '/Users/Jake/Documents/GitHub/all-sports-win-tracker/win_tracker/soccer/epl_final.csv'  
prior_data = spark.read.csv(py_csv_file_path, header=True, inferSchema=True,).withColumnRenamed("home_team", "HomeTeam").withColumnRenamed("away_team", "AwayTeam" )
prior_data.show()


+-------+----------+-------------+-------------+-----------------+-----------------+--------------+-----------------+-----------------+--------------+---------+---------+-----------------+-----------------+-----------+-----------+---------+---------+---------------+---------------+------------+------------+
| Season| MatchDate|     HomeTeam|     AwayTeam|FullTimeHomeGoals|FullTimeAwayGoals|FullTimeResult|HalfTimeHomeGoals|HalfTimeAwayGoals|HalfTimeResult|HomeShots|AwayShots|HomeShotsOnTarget|AwayShotsOnTarget|HomeCorners|AwayCorners|HomeFouls|AwayFouls|HomeYellowCards|AwayYellowCards|HomeRedCards|AwayRedCards|
+-------+----------+-------------+-------------+-----------------+-----------------+--------------+-----------------+-----------------+--------------+---------+---------+-----------------+-----------------+-----------+-----------+---------+---------+---------------+---------------+------------+------------+
|2000/01|2000-08-19|     Charlton|     Man City|                4|       

In [5]:
prior_data = (
    prior_data.withColumn("FullTimeHomeGoals", col("FullTimeHomeGoals").cast("int"))
    .withColumn("FullTimeAwayGoals", col("FullTimeAwayGoals").cast("int"))
    .withColumn("HomeTeam", col("HomeTeam").cast("string"))
    .withColumn("AwayTeam", col("AwayTeam").cast("string"))
)

In [6]:
prior_data = prior_data.select(
    "HomeTeam",
    "AwayTeam",
    "FullTimeHomeGoals",
    "FullTimeAwayGoals",
    "Season",
    "MatchDate",
)

In [7]:
gw_data = (
    spark.read.csv(
        f"/Users/Jake/Documents/GitHub/FPL-Elo-Insights/data/2025-2026/By Gameweek/GW*/matches.csv",
        header=True,
        inferSchema=True,
    ).filter(col("tournament") == "prem")
    .select(
        "gameweek",
        "kickoff_time",
        "home_team",
        "home_team_elo",
        "home_score",
        "away_score",
        "away_team",
        "away_team_elo",
        "finished",
        "match_id",
        "tournament",
    )
    .withColumn("Season", col("kickoff_time").substr(1, 4))
    .withColumn("MatchDate", col("kickoff_time").substr(1, 10))
)
spark.sparkContext.setLogLevel("ERROR")


gw_data.show()

25/11/23 14:39:54 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: /Users/Jake/Documents/GitHub/FPL-Elo-Insights/data/2025-2026/By Gameweek/GW*/matches.csv.
java.io.FileNotFoundException: File /Users/Jake/Documents/GitHub/FPL-Elo-Insights/data/2025-2026/By Gameweek/GW*/matches.csv does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:917)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1238)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:907)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:56)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:381)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$ca

+--------+-------------------+---------+-------------+----------+----------+---------+-------------+--------+--------------------+----------+------+----------+
|gameweek|       kickoff_time|home_team|home_team_elo|home_score|away_score|away_team|away_team_elo|finished|            match_id|tournament|Season| MatchDate|
+--------+-------------------+---------+-------------+----------+----------+---------+-------------+--------+--------------------+----------+------+----------+
|    11.0|2025-11-08 17:30:00|     56.0|      1642.51|       2.0|       2.0|      3.0|       2046.1|    true|25-26-prem-sunder...|      prem|  2025|2025-11-08|
|    11.0|2025-11-08 20:00:00|      8.0|      1902.65|       3.0|       0.0|     39.0|      1686.77|    true|25-26-prem-chelse...|      prem|  2025|2025-11-08|
|    11.0|2025-11-23 14:00:00|      2.0|         NULL|       1.0|       2.0|      7.0|      1886.16|    true|25-26-prem-leeds-...|      prem|  2025|2025-11-23|
|    11.0|2025-11-09 14:00:00|     94.0|

In [8]:
from pyspark.sql.types import StringType

gw_data = (
    gw_data.withColumn("FullTimeHomeGoals", col("home_score").cast("int"))
    .withColumn("FullTimeAwayGoals", col("away_score").cast("int"))
)

gw_data.show()

+--------+-------------------+---------+-------------+----------+----------+---------+-------------+--------+--------------------+----------+------+----------+-----------------+-----------------+
|gameweek|       kickoff_time|home_team|home_team_elo|home_score|away_score|away_team|away_team_elo|finished|            match_id|tournament|Season| MatchDate|FullTimeHomeGoals|FullTimeAwayGoals|
+--------+-------------------+---------+-------------+----------+----------+---------+-------------+--------+--------------------+----------+------+----------+-----------------+-----------------+
|    11.0|2025-11-08 17:30:00|     56.0|      1642.51|       2.0|       2.0|      3.0|       2046.1|    true|25-26-prem-sunder...|      prem|  2025|2025-11-08|                2|                2|
|    11.0|2025-11-08 20:00:00|      8.0|      1902.65|       3.0|       0.0|     39.0|      1686.77|    true|25-26-prem-chelse...|      prem|  2025|2025-11-08|                3|                0|
|    11.0|2025-11-23

In [9]:
team_codes = {
    3.0: "Arsenal",
    7.0: "Aston Villa",
    90.0: "Burnley",
    91.0: "Bournemouth",
    94.0: "Brentford",
    36.0: "Brighton",
    21.0: "West Ham",
    39.0: "Wolves",
    8.0: "Chelsea",
    31.0: "Crystal Palace",
    11.0: "Everton",
    54.0: "Fulham",
    2.0: "Leeds",
    14.0: "Liverpool",
    43.0: "Man City",
    1.0: "Man Utd",
    4.0: "Newcastle",
    17.0: "Nott'm Forest",
    56.0: "Sunderland",
    6.0: "Spurs",
}

gw_data = map_teams(
    gw_data,
    team_codes,
    "home_team",
    "away_team",
)
gw_data.show()

+--------+-------------------+--------------+-------------+----------+----------+-----------+-------------+--------+--------------------+----------+------+----------+-----------------+-----------------+
|gameweek|       kickoff_time|     home_team|home_team_elo|home_score|away_score|  away_team|away_team_elo|finished|            match_id|tournament|Season| MatchDate|FullTimeHomeGoals|FullTimeAwayGoals|
+--------+-------------------+--------------+-------------+----------+----------+-----------+-------------+--------+--------------------+----------+------+----------+-----------------+-----------------+
|    11.0|2025-11-08 17:30:00|    Sunderland|      1642.51|       2.0|       2.0|    Arsenal|       2046.1|    true|25-26-prem-sunder...|      prem|  2025|2025-11-08|                2|                2|
|    11.0|2025-11-08 20:00:00|       Chelsea|      1902.65|       3.0|       0.0|     Wolves|      1686.77|    true|25-26-prem-chelse...|      prem|  2025|2025-11-08|                3|    

In [10]:
full_data = prior_data.unionByName(gw_data, allowMissingColumns=True)

full_data = full_data.withColumn("day_of_week_name", date_format("MatchDate", "EEEE"))
full_data.show()
full_data.count()

+-------------+-------------+-----------------+-----------------+-------+----------+--------+------------+---------+-------------+----------+----------+---------+-------------+--------+--------+----------+----------------+
|     HomeTeam|     AwayTeam|FullTimeHomeGoals|FullTimeAwayGoals| Season| MatchDate|gameweek|kickoff_time|home_team|home_team_elo|home_score|away_score|away_team|away_team_elo|finished|match_id|tournament|day_of_week_name|
+-------------+-------------+-----------------+-----------------+-------+----------+--------+------------+---------+-------------+----------+----------+---------+-------------+--------+--------+----------+----------------+
|     Charlton|     Man City|                4|                0|2000/01|2000-08-19|    NULL|        NULL|     NULL|         NULL|      NULL|      NULL|     NULL|         NULL|    NULL|    NULL|      NULL|        Saturday|
|      Chelsea|     West Ham|                4|                2|2000/01|2000-08-19|    NULL|        NULL|  

9760

In [11]:
full_data_soccer = add_winner_column(
    full_data,
    home_goals_col="FullTimeHomeGoals",
    away_goals_col="FullTimeAwayGoals",
    home_team_col="HomeTeam",
    away_team_col="AwayTeam",
)


full_data_soccer.show()
full_data_soccer

+-------------+-------------+-----------------+-----------------+-------+----------+--------+------------+---------+-------------+----------+----------+---------+-------------+--------+--------+----------+----------------+-------------+
|     HomeTeam|     AwayTeam|FullTimeHomeGoals|FullTimeAwayGoals| Season| MatchDate|gameweek|kickoff_time|home_team|home_team_elo|home_score|away_score|away_team|away_team_elo|finished|match_id|tournament|day_of_week_name|       Winner|
+-------------+-------------+-----------------+-----------------+-------+----------+--------+------------+---------+-------------+----------+----------+---------+-------------+--------+--------+----------+----------------+-------------+
|     Charlton|     Man City|                4|                0|2000/01|2000-08-19|    NULL|        NULL|     NULL|         NULL|      NULL|      NULL|     NULL|         NULL|    NULL|    NULL|      NULL|        Saturday|     Charlton|
|      Chelsea|     West Ham|                4|     

DataFrame[HomeTeam: string, AwayTeam: string, FullTimeHomeGoals: int, FullTimeAwayGoals: int, Season: string, MatchDate: date, gameweek: double, kickoff_time: timestamp, home_team: string, home_team_elo: double, home_score: double, away_score: double, away_team: string, away_team_elo: double, finished: boolean, match_id: string, tournament: string, day_of_week_name: string, Winner: string]